In [1]:
# Require Pandas version >= 2.2.2
import pandas as pd
assert pd.__version__ >= '2.2.2', 'Please update Pandas to version 2.2.2 or newer'
import pandas as pd, os, pyreadr
import warnings
import time
import numpy as  np
import pandas as pd
import pandas as pd
import visualization as viz
import data_loading as dl
import risk_pricing as rp
import statsmodels.api as sm
import statsmodels.formula.api as smf
import importlib
importlib.reload(rp)
importlib.reload(viz)
importlib.reload(dl)
warnings.filterwarnings("ignore")
source_folder_path = "/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/"
modelname = "embeddings_km10_ipcs" # "dicfullmc10thr10defnob40noa0_8_hdp" # Don't forget to clean again when modelname changes
start_time = time.time()
# print the current working directory (for reference)
print("Current Working Directory:", os.getcwd())
os.chdir(source_folder_path)
print("New Working Directory:", os.getcwd())

Current Working Directory: /Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src
New Working Directory: /Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src


In [2]:
# # Creating additional models

# embeddings = Embeddings()
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_knocap", "knowledge capital", to_csv = True)
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_patents", "patents", to_csv = True)
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_intcap", "intangible capital", to_csv = True)

In [3]:
# Loading the data from load_dataframes:


In [6]:
import risk_pricing as rp
import importlib
import embedding_tools as et
from embedding_tools import Embeddings
from embedding_tools import EmbeddingsHKRModel
importlib.reload(rp)
importlib.reload(viz)
quantiles = 10
embeddings = Embeddings()
EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_kt", "key talent", to_csv = True)
for modelname in ["emb_kt"]:
    for pfname in ['pfkk2me3mb', 'pfkki3me3mb']:
        for quantiles in [10]:
            amazon_nov01_short, cequity_mapper, ff3fw, ff5fw, ff3fm, ff5fm, topic_map_unlabeled, comparison_measures, stoxmo_orig, stoxda_orig, stoxwe_orig = \
                dl.load_dataframes(modelname, start_time, clean_again = True)
            cuts = np.linspace(0, 1, quantiles+1).tolist()
            topic_map = viz.label_topic_map(topic_map_unlabeled, modelname, cuts = cuts)
            fmb_5, indexed_df, eret_we, stoxwe_add = rp.famaMacBethFull(stoxwe_orig, cequity_mapper, topic_map, ff5fw, pfname, kki_cuts = cuts, window_size = 52*2, add_innerkk_pf = True)
            fmb_3, _, _, _ = rp.famaMacBethFull(stoxwe_orig, cequity_mapper, topic_map, ff3fw, pfname, kki_cuts = cuts, window_size = 52*2, add_innerkk_pf = True)
            fmb_1, _, _, _ = rp.famaMacBethFull(stoxwe_orig, cequity_mapper, topic_map, ff3fw, pfname, formula = "eretw ~ MktRF + HKR", kki_cuts = cuts, window_size = 52*2, add_innerkk_pf = True)
            #import visualization as viz
            base_path = "/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/text/"
            dir_path = os.path.join(base_path, f"{modelname}_{quantiles}tiles_{pfname}")
            os.makedirs(dir_path, exist_ok=True)
            figfolder = os.path.join(dir_path, "")
            generate_plots = True
            ##
            viz.explore_topic_map(topic_map, figfolder, generate_plots)
            viz.explore_fmb([fmb_1, fmb_3, fmb_5], figfolder)
            viz.explore_eret_we(eret_we, figfolder)
            viz.tex_compare_kk_measures(comparison_measures, figfolder)
            viz.plot_returns(stoxwe_add, figfolder)
            viz.amazon_graph(amazon_nov01_short, figfolder)
            viz.filecounter(figfolder)
            viz.explore_stoxda(stoxda_orig, cequity_mapper, topic_map, figfolder)

Embeddings loaded from data/embeddings.pkl
Tempo de execução: 10649.735467910767


/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/data_loading/data_loading.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


Tempo de execução: 10652.5975420475
Tempo de execução: 10652.630290031433
Tempo de execução: 10652.66195487976
Tempo de execução: 10652.95087814331
Tempo de execução: 10676.664582967758


/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/risk_pricing/risk_pricing.py:398: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/risk_pricing/risk_pricing.py:424: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/risk_

Executing fig_mean_tiy...
Executing tex_average_topic_loadings_by_high_tech...
Executing tex_sample_topic_loadings...
Executing df_skill_correlations...
Executing df_patent_correlations...
Executing fig_share_dominant_kk_by_ind...
Executing fig_heatmap_topicvskkpt...
Executing fig_heatmap_topicvsikpt...
Executing fig_histogram_kk...
Executing fig_histogram_kk_by_ind12...
Finished!
Executing explore_fmb...
Executing tex_summary...
Executing fig_h1b_vs_smb_kkhml...
Executing tex_HKR_vs_mktrf...
Executing tex_summary_statistics...
Now with Sharpe ratio
Executing tex_compare_kk_measures...
Executing plot_returns...


...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [ ]:
stoxwe_add.columns

Index(['yw', 'PERMNO', 'retw', 'y', 'gvkey', 'cusip8', 'cik_x', 'year_x',
       'crit_EXCHG', 'crit_PRC', 'crit_SHRCD', 'crit_ALL', 'max_topic',
       'topic_kk', 'year_y', 'CIK', 'CUSIP8', 'LPERMNO', 'sic', 'cik_y',
       'gvkey_y', 'conm', 'naics4', 'ind12', 'sic3', 'hi_tech', 'Skill',
       'xi_yeartotal', 'xi_cumsum', 'K_int_Know', 'K_int', 'at', 'prcc_f',
       'prcc_c', 'ppegt', 'csho', 'ceq', 'cusip', 'exchg', 'xir_cumsum',
       'xir_total', 'ntile_kk', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF',
       'eretw', 'fiscalyear', 'mb', 'me', 'kk_share', 'pfkk3me3mb', 'pf2me3mb',
       'pf5me5mb', 'pfkk2me3mb', 'pfkki3me3mb', 'me_group'],
      dtype='object')

In [ ]:
max_kknt = 7
HKR_ret = (stoxwe_add
            .loc[:, ['yw', 'ntile_kk', 'me_group', 'topic_kk', 'eretw', 'me']]
            .dropna(subset=['topic_kk'])
            .groupby(['yw', 'ntile_kk', 'me_group'])
            .apply(lambda df: pd.Series({
                'eret': (df['eretw'] * df['me']).sum() / df['me'].sum()}))
            .reset_index()
            .groupby(['yw', 'ntile_kk'])
            .agg(eret_mean=('eret', 'mean'))
            .reset_index()
            .pivot_table(index='yw', columns='ntile_kk', values='eret_mean', aggfunc='mean')
            .rename(columns=lambda x: f'kk{x}')
            .assign(HKR_SB=lambda df: df[f'kk{max_kknt}'] - df['kk0'])
            .reset_index()
            .loc[:, ['yw', 'HKR_SB']])
HKR_ret.columns.name = None
HKR_ret 
# HKR_ret = (stoxwe_add.dropna(subset=['topic_kk'])
#             .groupby(['yw', 'ntile_kk'])
            # .apply(lambda df: pd.Series({
            #     'eret': (df['eretw'] * df['me']).sum() / df['me'].sum()}))
#             .reset_index()

,yw,HKR_SB
0,2006-07-03,-0.028851
1,2006-07-10,-0.039941
2,2006-07-17,-0.015079
3,2006-07-24,0.025082
4,2006-07-31,-0.019042
...,...,...
750,2020-12-07,0.014406
751,2020-12-14,-0.001068
752,2020-12-21,0.017294
753,2020-12-28,0.006375


In [ ]:
# dog = (stoxwe_add.dropna(subset=['topic_kk'])
#              .groupby(['yw', 'ntile_kk', 'me_group'])
#              .apply(lambda df: pd.Series({
#                  'eret': (df['eretw'] * df['me']).sum() / df['me'].sum()})))
stoxwe_add.columns
# HKR_ret = (stoxwe_add.dropna(subset=['topic_kk'])
#             .groupby(['yw', 'ntile_kk', 'me_group'])
#             .apply(lambda df: pd.Series({
#                 'eret': (df['eretw'] * df['me']).sum() / df['me'].sum()}))
#             .reset_index()
#             .pivot_table(index='yw', columns='ntile_kk', values='eret', aggfunc='mean')
#             .rename(columns=lambda x: f'kk{x}')
#             .assign(HKR=lambda df: df[f'kk{max_kknt}'] - df['kk0'])
#             [['HKR']]
#             .reset_index())

Index(['yw', 'PERMNO', 'retw', 'y', 'gvkey', 'cusip8', 'cik_x', 'year_x',
       'crit_EXCHG', 'crit_PRC', 'crit_SHRCD', 'crit_ALL', 'max_topic',
       'topic_kk', 'year_y', 'CIK', 'CUSIP8', 'LPERMNO', 'sic', 'cik_y',
       'gvkey_y', 'conm', 'naics4', 'ind12', 'sic3', 'hi_tech', 'Skill',
       'xi_yeartotal', 'xi_cumsum', 'K_int_Know', 'K_int', 'at', 'prcc_f',
       'prcc_c', 'ppegt', 'csho', 'ceq', 'cusip', 'exchg', 'xir_cumsum',
       'xir_total', 'ntile_kk', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF',
       'eretw', 'fiscalyear', 'mb', 'me', 'kk_share', 'pfkk3me3mb', 'pf2me3mb',
       'pf5me5mb', 'pfkk2me3mb', 'pfkki3me3mb', 'me_group'],
      dtype='object')

Variables created:

- amazon_nov01_short:

- cequity_mapper:

- Fama-French factors: ff3fw, ff5fw, ff3fm, ff5fm

- Topic_map_unlabeled

- Stocks vectors: stoxmo_orig, stoxwe_orig

- comparison_measures: a vector that calculates the average industry skill and firm patent intensity for each of the topics.